The idea of that notebook is to have a reproducible benchmark working with the masters version of `pysap` and `modopt` in April 2019.
The benchmark will be on speed, for both sparkling and uniform random acquisition schemes, with a brain phantom, for decimated wavelets.

The benchmark will future the algorithms of the abstract submission for SPARS 2019.

Special attention must be taken for Condat. We might need to qualify the benchmark with non reproducible experiments where we set the norm of the linear operator.
We also need to take care of the relaxation factor.

## Install dependencies

***On Google Colab, it is required to install the following dependencies in order for the notebook to work***

In [ ]:
!pip install mri-nufft[finufft,gpunufft] pyWavelets pysap-mri brainweb-dl "numpy<2.0" python-pysap 

In [ ]:
# We will also dowload extra data if needed:

In [ ]:
%load_ext autoreload
%autoreload 2

# Third party import
import matplotlib.pyplot as plt
import numpy as np
# from scipy.ndimage import imread
# from tqdm import tqdm_notebook

# Package import
from modopt.math.metrics import ssim
import pysap
from pysap.data import get_sample_data
from mri.operators.utils import convert_locations_to_mask, gridded_inverse_fourier_transform_nd
from mri.operators import NonCartesianFFT, WaveletUD2
from mri.reconstructors import SelfCalibrationReconstructor, SingleChannelReconstructor
from modopt.opt.proximity import SparseThreshold, GroupLASSO
from modopt.opt.linear import Identity

#  Loading input data

In [ ]:
# Loading input data
image = get_sample_data('2d-mri')

# Obtain MRI non-cartesian mask
radial_mask = get_sample_data("mri-radial-samples")
kspace_loc = radial_mask.data
mask = pysap.Image(data=convert_locations_to_mask(kspace_loc, image.shape))
plt.imshow(mask, cmap='gray')

# Generate the kspace
From the 2D brain slice and the acquistion mask, we generate the acquisition measurments, the observed kspace. We then reconstruct the zero order solution.

In [ ]:
fourier_op = NonCartesianFFT(samples=kspace_loc, shape=image.shape, implementation='gpuNUFFT')
kspace_obs = fourier_op.op(image.data)

Zero order solution

In [ ]:
grid_space = np.linspace(-0.5, 0.5, num=image.shape[0])
grid2D = np.meshgrid(grid_space, grid_space)
grid_soln = gridded_inverse_fourier_transform_nd(kspace_loc, kspace_obs,
                                                 tuple(grid2D), 'linear')
image_rec0 = pysap.Image(data=grid_soln)
# image_rec0.show()
base_ssim = ssim(image_rec0, image)
print('The Base SSIM is : ' + str(base_ssim))

# FISTA optimization
We now want to refine the zero order solution using a FISTA optimization.

In [ ]:
mu = 2 * 1e-7

In [ ]:
linear_op = WaveletUD2(
    wavelet_id=24,
    nb_scale=4,
)
regularizer_op = SparseThreshold(Identity(), mu, thresh_type="soft")
# Setup Reconstructor
reconstructor = SingleChannelReconstructor(
    fourier_op=fourier_op,
    linear_op=linear_op,
    regularizer_op=regularizer_op,
    gradient_formulation='synthesis',
    verbose=1,
)

In [ ]:
def objective_cost(x):
    return data_fidelity(x) + sparsity(x)

def sparsity(x):
    return mu * np.sum(np.abs(x))

def data_fidelity(x):
    return 0.5 * np.linalg.norm(fourier_op.op(x) - kspace_obs)**2

In [ ]:
def nrmse(x):
    return np.linalg.norm(x - image) / np.mean(image)

In [ ]:
metrics_ = {
    "cost": {"metric": objective_cost, "mapping": {"z_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
    "nrmse": {"metric": nrmse, "mapping": {"x_new": "x"}, "cst_kwargs": {}, "early_stopping": False},
}

In [ ]:
opt_results = {}

In [ ]:
x_final, costs, metrics = reconstructor.reconstruct(
    kspace_data=kspace_obs,
    optimization_alg='fista',
    num_iterations=200,
    metrics=metrics_,
    metric_call_period=1,
)
image_rec = pysap.Image(data=np.abs(x_final))

In [ ]:
opt_results[mu] = {
        "nrmse": metrics['nrmse']['values'][-1],
        "cost": metrics['cost']['values'][-1],
    }
print(opt_results)

In [ ]:
params = {
    r"FISTA-BT": ({}, "black"),
    r"FISTA-CD, $a = 20$": ({"a_cd": 20}, "blue"),
    r"Rada-FISTA": (
        {"p_lazy": (1/30), "q_lazy": (1/10), "restart_strategy": "adaptive", "xi_restart": 0.96},
        "orange",
    ),
    r"greedy FISTA": (
        {"restart_strategy": "greedy", "xi_restart": 0.96, "s_greedy": 1.1},
        "violet",
    ),
}

res_param = {}
for param_name, (param, _) in (params.items()):
    x_final, costs, metrics = reconstructor.reconstruct(
        kspace_data=kspace_obs,
        optimization_alg='fista',
        num_iterations=200,
        metrics = metrics_,
        metric_call_period=1,
        **param,
    )
    res_param[param_name] = {
        "cost": np.array(metrics['cost']['values']),
        "nrmse": np.array(metrics['nrmse']['values']),
    }

In [ ]:
metric_name = "cost"
plt.figure(figsize=(9, 5))
for param_name, param_res in res_param.items():
    plt.plot(
        np.log10(np.abs(np.array(param_res[metric_name] - opt_results[mu][metric_name]))),
        label=param_name,
        color=params[param_name][1],
    )
plt.legend()
plt.title(r"Cost evolution for different restarting strategies")
plt.xlabel(r"$k$")
plt.ylabel(r"$\log_{10}($cost$(k) -$ cost$*)$")
plt.show()
plt.show(block = False)

In [ ]:

import matplotlib.pyplot as plt

# Define Data

x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]

# Plot

plt.plot(x, y, color='green', linewidth=3, linestyle='dotted')

# Display

plt.show(block=True)

# Print Statement

print('My First Matplotlib Program')